In [1]:
TRACE_PATH = 'traces/trace-blueline-QP1A.190711.019-2024-03-04-00-43-57.perfetto-trace'


1. developer (click 10 time)
2. "Developer options" --> "System Tracing" --> "long traces" & "Show Quick Setting title"
3. manually open the system tracing by click the tool && run-once.py
4. manually close the system tracing when the "run-once.py" finish

In [2]:
# We use perfetto to get the cpu utilization
from perfetto.trace_processor import TraceProcessor


def AnalysisJankAndFPS(process_name, trace_processor, debug=False):
    # SQL = 'select ts, dur, surface_frame_token as app_token, display_frame_token as sf_token, process.name \
    #         from expected_frame_timeline_slice left join process using(upid)'

    # SQL = 'select ts, dur, surface_frame_token as app_token, display_frame_token, jank_type,\
    #         on_time_finish, present_type, layer_name, process.name \
    #         from actual_frame_timeline_slice left join process using(upid)'

    # SQL = 'select id, type, ts, dur, track_id, category, name, depth, stack_id, \
    #        parent_stack_id, parent_id, arg_set_id, cat, slice_id \
    #        from slice \
    #        where name like \'%doFrame%\' \
    #        order by dur desc;'


    # SQL = 'select slice_id, track_id, ts, dur, dur/1e6, slice.name as slice_name, process.name as process_name \
    #        from slice \
    #        join thread_track \
    #        on slice.track_id = thread_track.id \
    #        join thread using(utid) \
    #        join process using(upid) \
    #        where process.name = \'com.amazon.mShop.android.shopping\' and slice.name like \'%doFrame%\' \
    #        order by ts desc;'


    SQL = 'select slice_id, track_id, ts, dur, dur/1e6, slice.name as slice_name, process.name as process_name \
           from slice \
           join thread_track \
           on slice.track_id = thread_track.id \
           join thread using(utid) \
           join process using(upid) \
           where process.name = \'%s\' and slice.name = \'Choreographer#doFrame\' \
           order by ts desc;' % process_name

    # qr_it = trace_processor.query(SQL)
    # for idx, row in enumerate(qr_it):
    #     if idx <= 5:
    #         print(row)

    qr_it = trace_processor.query(SQL)
    data = []
    for idx, row in enumerate(qr_it):
        row = '%s' % row
        row = eval(row)
        data.append(row)
    ts_arr = [x['ts'] for x in data]
    ts_arr = sorted(ts_arr)

    ts_delta_arr = []
    for i in range(1, len(ts_arr)):
        delta = ts_arr[i] - ts_arr[i-1]
        ts_delta_arr.append(delta)


    ts_delta_arr = sorted(ts_delta_arr, reverse=True)

    jank_num = 0
    all_num = len(ts_delta_arr)

    for t in ts_delta_arr:
    #     if t/1e9 > 1/60.0:
        if t/1e6 > 16.7:
            jank_num += 1

    if all_num == 0:
        jank_rate = float('inf')
    else:
        jank_rate = jank_num/all_num
        
    if len(ts_arr) <= 1:
        fps = 0
    else:
        fps = len(ts_arr) / ((ts_arr[-1] - ts_arr[0]) / 1e9)
    
    
    if debug:
        print()
        print('Largest 100 delta: ', ts_delta_arr[:100])
        print('Smallest 100 delta: ', ts_delta_arr[-100:-1])
        
        print()
        print('jank_num=%d, all_num=%d, jank_rate=%f' % (jank_num, all_num, jank_rate))
        print('fps=%f' % fps)
    
    return jank_rate, fps



def AnalysisCPU(process_name, trace_processor, debug=False):
    # Perfetto: thread sorted by cpu_sec
    # SQL = 'select * from sched'

    # SQL = 'select * from thread'

    # SQL = 'select * from process'

    # SQL = 'select process.name as process, thread.name as thread, cpu, sum(dur) as cpu_dur \
    #         from sched \
    #         inner join thread using(utid) \
    #         inner join process using(upid) group by utid, cpu order by cpu_dur desc'

    SQL = 'select process.name as process, thread.name as thread, sum(dur) as cpu_dur \
            from sched \
            inner join thread using(utid) \
            inner join process using(upid) group by utid order by cpu_dur desc'

    # SQL = 'select process.name as process, thread.name as thread, cpu, sum(dur) as cpu_dur \
    #         from sched \
    #         inner join thread using(utid) \
    #         inner join process using(upid) group by process order by cpu_dur desc'


    qr_it = trace_processor.query(SQL)

    all_runtime_ns = 0
    app_runtime_ns = 0
    app_mutator_runtime_ns = 0
    app_gc_runtime_ns = 0
    all_mutator_runtime_ns = 0
    all_gc_runtime_ns = 0



    for idx, row in enumerate(qr_it):
        row = '%s' % row
        row = eval(row)

    #     if idx >= 100:
    #         break
    #     print(row)

        process = row['process']
        thread = row['thread']
        dur = row['cpu_dur']

        all_runtime_ns += dur

        if process == process_name:
            app_runtime_ns += dur
            if thread == 'HeapTaskDaemon':
                app_gc_runtime_ns += dur
            else:
                app_mutator_runtime_ns += dur


        if thread == 'HeapTaskDaemon':
            all_gc_runtime_ns += dur
        elif process != None:
            all_mutator_runtime_ns += dur

    app_runtime = app_runtime_ns / 1e9
    app_mutator_runtime = app_mutator_runtime_ns / 1e9
    app_gc_runtime = app_gc_runtime_ns / 1e9
            
    all_runtime = all_runtime_ns / 1e9
    all_mutator_runtime = all_mutator_runtime_ns / 1e9
    all_gc_runtime = all_gc_runtime_ns / 1e9
    
    if debug:
        print('app_runtime=%f' % (app_runtime))
        print('app_mutator_runtime=%f' % (app_mutator_runtime))
        print('app_gc_runtime=%f' % (app_gc_runtime))

        print()
        print('all_runtime=', all_runtime)
        print('all_mutator_runtime=%f' % (all_mutator_runtime))
        print('all_gc_runtime=%f' % (all_gc_runtime))

    return app_runtime, app_mutator_runtime, app_gc_runtime, all_runtime, all_mutator_runtime, all_gc_runtime


def AnalsysTraceProcessor(app_list, trace_processor, debug=False):
    jank_res = {}
    fps_res= {}
    cpu = {}
    mutator_cpu = {}
    gc_cpu = {}

    for process_name in app_list:
        if debug:
            print(process_name)
        jank_rate, fps = AnalysisJankAndFPS(process_name, trace_processor, debug)
        if debug:
            print('jank_rate=%f, fps=%f' % (jank_rate, fps))

        app_runtime, app_mutator_runtime, app_gc_runtime, all_runtime, all_mutator_runtime, all_gc_runtime = AnalysisCPU(process_name, trace_processor, debug)
        if debug:
            print('app_runtime=%f, app_mutator_runtime=%f, app_gc_runtime=%f, all_runtime=%f, all_mutator_runtime=%f, all_gc_runtime=%f' %\
                 (app_runtime, app_mutator_runtime, app_gc_runtime, all_runtime, all_mutator_runtime, all_gc_runtime))
            print()
        
        jank_res[process_name] = jank_rate
        fps_res[process_name] = fps
        cpu[process_name] = app_runtime / all_runtime * len(app_list)
        mutator_cpu[process_name] = app_mutator_runtime / all_runtime * len(app_list)
        gc_cpu[process_name] = app_gc_runtime / all_runtime * len(app_list)
    return jank_res, fps_res, cpu, mutator_cpu, gc_cpu

    


In [3]:
app_list = [
    'com.twitter.android',
    'com.facebook.katana',
    'com.instagram.android',
    'org.telegram.messenger',
    'jp.naver.line.android',
    
    'com.google.android.youtube',
    'com.ss.android.ugc.aweme',
    'com.spotify.music',
    'tv.twitch.android.app',
    'com.wemesh.android',
    'sg.bigo.live',

    'com.amazon.mShop.android.shopping',
    'com.google.android.apps.maps',
    'com.android.chrome',
    'org.mozilla.firefox',
    'com.linkedin.android',

    'com.rovio.angrybirds',
    'com.king.candycrushsaga',
]


trace_processor = TraceProcessor(trace=TRACE_PATH)

print('TraceProcessor Finished! \nTRACE_PATH=%s' % TRACE_PATH)



jank, fps, cpu, mutator_cpu, gc_cpu = AnalsysTraceProcessor(app_list, trace_processor)

print('\nJank:')
for x in app_list:
    print(x, jank[x])
    
print('\n FPS:')
for x in app_list:
    print(x, fps[x])
    
print('\n CPU:')
for x in app_list:
    print(x, cpu[x])
    
print('\n CPU mutator:')
for x in app_list:
    print(x, mutator_cpu[x])
    
print('\n CPU GC:')
for x in app_list:
    print(x, gc_cpu[x])
    

jank = [jank[x] for x in app_list]
fps = [fps[x] for x in app_list]
cpu = [cpu[x] for x in app_list]
mutator_cpu = [mutator_cpu[x] for x in app_list]
gc_cpu = [gc_cpu[x] for x in app_list]




TraceProcessor Finished! 
TRACE_PATH=traces/trace-blueline-QP1A.190711.019-2024-03-04-00-43-57.perfetto-trace

Jank:
com.twitter.android 0.09767441860465116
com.facebook.katana 0.0755287009063444
com.instagram.android 0.11393805309734513
org.telegram.messenger 0.22641509433962265
jp.naver.line.android 0.06259541984732825
com.google.android.youtube 0.06431852986217458
com.ss.android.ugc.aweme 0.13712374581939799
com.spotify.music 0.11842105263157894
tv.twitch.android.app 0.045925925925925926
com.wemesh.android 0.17752100840336135
sg.bigo.live 0.11252268602540835
com.amazon.mShop.android.shopping 0.07876712328767123
com.google.android.apps.maps 0.0881542699724518
com.android.chrome 0.08818635607321132
org.mozilla.firefox inf
com.linkedin.android 0.11332312404287902
com.rovio.angrybirds 0.1431980906921241
com.king.candycrushsaga 0.11294765840220386

 FPS:
com.twitter.android 37.51165287665345
com.facebook.katana 23.765251969178596
com.instagram.android 56.72208449692362
org.telegram.messe